In [ ]:
# installs dependencies and imports required libraries.
%pip install -U transformers

# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("question-answering", model="deepset/tinyroberta-squad2")

In [ ]:
import pandas as pd
from tqdm import tqdm
import gc # Import garbage collector module

# 1. Import dataset
try:
    df = pd.read_csv('../data/BDFoodSent-334k.csv', encoding='utf-8')
except FileNotFoundError:
    print("Error: Please check file name...")
    exit()

REVIEW_COLUMN = 'text' # Ensure column name is correct
if REVIEW_COLUMN not in df.columns:
    print(f"Error: There is no '{REVIEW_COLUMN}' column in your data set...")
    exit()

# 2. Define aspects and questions for extraction
aspect_questions = {
    'Taste_Aspect': 'What is the opinion about the taste, quantity or food quality?',
    'Service_Aspect': 'What is the feedback on the service or delivery?',
    'Ambiance_Aspect': 'What is the description of the restaurant\'s atmosphere or environment?'
}

# 3. Set Batch Size
# This is crucial. The number of reviews processed in each batch.
# Start with a small number, e.g., 50 or 100. Decrease if memory crashes continue.
BATCH_SIZE = 20

# 4. Initialize an empty list to store results from all batches
results_list = []

# 5. Loop and process data in batches
print(f"Starting batch processing with batch size: {BATCH_SIZE}")
for i in tqdm(range(0, len(df), BATCH_SIZE), desc="Processing Batches"):
    # Determine the start and end index for the current batch
    batch_start = i
    batch_end = i + BATCH_SIZE

    # Extract the reviews for the current batch
    # Use loc for reliable slicing
    batch_reviews = df.loc[batch_start:batch_end-1, REVIEW_COLUMN].tolist()

    # Initialize a list of dictionaries to store results for the current batch
    batch_results = []

    # Process each review within the batch
    for review in batch_reviews:
        row_result = {}
        for aspect, question in aspect_questions.items():
            try:
                # Use the previously loaded 'pipe' model
                res = pipe({
                    'question': question,
                    'context': review
                })
                row_result[aspect + '_Answer'] = res['answer']
                row_result[aspect + '_Score'] = res['score']
            except Exception as e:
                # Handle any errors during model inference
                row_result[aspect + '_Answer'] = f"Error: {e}"
                row_result[aspect + '_Score'] = 0.0

        batch_results.append(row_result)

    # Append current batch results to the master results list
    results_list.extend(batch_results)

    # Force garbage collection to free up memory
    gc.collect()

# 6. Convert the results list to a DataFrame
extracted_df = pd.DataFrame(results_list)

# 7. Merge the extracted results with the original data (ensure equal length)
# Use df.reset_index(drop=True) to ensure a clean merge on index
final_df = pd.concat([df.reset_index(drop=True), extracted_df], axis=1)

# 8. Export results to a new CSV file (same as before)
OUTPUT_FILENAME = 'analyzed_reviews_batch_output.csv'
final_df.to_csv(OUTPUT_FILENAME, index=False, encoding='utf-8')

print("\n--- Processing Complete ---")
print(f"Results successfully saved to file: {OUTPUT_FILENAME}")

Note: After the QA model is applied, the resulting new dataset consists of six additional columns representing [aspect + '_Answer'] and [aspect + '_Score']. Here, the aspect includes Taste, Service, and Ambiance, and the score represents the model's confidence score for the answer.

*** However, since the confidence score cannot objectively reflect the relevance between the answer and the question, a semantic relevance model is introduced below for further analysis. ***

In [ ]:
%pip install sentence-transformers
from sentence_transformers import SentenceTransformer, util
import torch
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

# Load Google Drive
from google.colab import drive
drive.mount('/content/drive')
print("Google Drive load done.")

In [ ]:
import pandas as pd
import numpy as np
import os, gc
from tqdm import tqdm

# --- Setup Paths (Assumed to be defined/loaded) ---
DRIVE_PATH = '/content/drive/MyDrive/'
PROJECT_FOLDER = 'Colab Notebooks/'

# 1. Define possible input file paths (Primary and Fallback)
PRIMARY_INPUT_FILE = os.path.join(DRIVE_PATH, PROJECT_FOLDER, 'analyzed_reviews_batch_output.csv')

# Output file for the Similarity Scores
OUTPUT_FILENAME_FINAL_MAX_DRIVE = os.path.join(DRIVE_PATH, PROJECT_FOLDER, 'final_analysis_with_similarity_wide.csv')

# 2. Define sub-topic questions and their main aspects
# NOTE: Reverting to the original single, composite question per main aspect for similarity calculation.
ALL_ASPECTS = {
    'Food_Aspect': [
        ('Composite', 'What is the opinion about the taste, quantity or food quality?')
    ],
    'Service_Aspect': [
        ('Composite', 'What is the feedback on the service or delivery?')
    ],
    'Ambiance_Aspect': [
        ('Composite', 'What is the description of the restaurant\'s atmosphere or environment?')
    ]
}

# 3. Define original answer columns corresponding to each main aspect (Keep unchanged)
ANSWER_COL_MAP = {
    'Food_Aspect': 'Taste_Aspect_Answer',
    'Service_Aspect': 'Service_Aspect_Answer',
    'Ambiance_Aspect': 'Ambiance_Aspect_Answer'
}

# 4. Robust Data Loading Block
try:
    # Attempt to load Primary file, include low_memory=False to solve parsing errors
    df_results = pd.read_csv(PRIMARY_INPUT_FILE, encoding='utf-8', low_memory=False)
    print(f"Data successfully loaded from: {PRIMARY_INPUT_FILE}")

except FileNotFoundError:
    print(f"FATAL: Input file not found at {PRIMARY_INPUT_FILE}. Please check your Drive path.")
    exit()
except Exception as e:
    print(f"FATAL: Failed to parse CSV file due to an error: {e}")
    exit()

# 5. Initialize new score storage
new_max_scores = {}
for aspect in ALL_ASPECTS.keys():
    new_max_scores[f'{aspect}_Similarity_Score'] = []

# 6. Batch Similarity Calculation (Revised: Prevents scores from being incorrectly set to 0 upon failure)
BATCH_SIZE = 512
print("Starting Final Similarity Calculation (3 Main Aspects)...")

# Track the count of rows where calculation failed (and score was set to 0)
failed_calculation_count = 0

for index, row in tqdm(df_results.iterrows(), total=len(df_results), desc="Calculating Max Similarity"):

    for main_aspect, sub_topics in ALL_ASPECTS.items():

        answer_col_name = ANSWER_COL_MAP[main_aspect]
        answer = row.get(answer_col_name)

        questions = [q for _, q in sub_topics]
        current_max_sim = 0.0 # Default score is 0.0

        # --- A. Check answer validity ---
        # Only enter calculation block if the answer is valid
        if not pd.isna(answer) and "Error:" not in str(answer) and answer != "":
            try:
                # Encode all sub-questions and the single answer in a batch
                sentences = questions + [answer]

                # ****** CRITICAL STEP: Model Calculation ******
                embeddings = model.encode(sentences, batch_size=BATCH_SIZE, convert_to_tensor=True).cpu()

                # Calculate similarity and find the maximum score
                q_embeddings = embeddings[:-1]
                a_embedding = embeddings[-1]
                sims = util.cos_sim(q_embeddings, a_embedding).flatten().tolist()

                current_max_sim = max(sims)
                # ****** If calculation succeeds, current_max_sim > 0 ******

            except Exception as e:
                # B. If calculation fails, record and keep 0.0
                current_max_sim = 0.0
                # Directly modify global counter (fix NameError)
                # Assuming this variable is at the top level of the script, we can access it directly
                global failed_calculation_count
                failed_calculation_count += 1
                # Only print failed row for diagnosing memory/data issues
                print(f"\nCalculation failed at index {index} for {main_aspect}: {e}")

        # Store the final MAX score in the dictionary
        new_max_scores[f'{main_aspect}_Similarity_Score'].append(current_max_sim)

    if index % 1000 == 0:
        gc.collect()

# Additional diagnostic information
total_attempts = len(df_results) * 3
print(f"\nTotal rows where calculation failed (and score was set to 0.0): {failed_calculation_count}")
print(f"Percentage of failed calculations: {failed_calculation_count / total_attempts * 100:.4f}%")
# 7. Merge the new MAX similarity score columns
df_new_scores = pd.DataFrame(new_max_scores)

# To prevent index misalignment during merge, we use reset_index
df_final = pd.concat([df_results.reset_index(drop=True), df_new_scores.reset_index(drop=True)], axis=1)

# 8. Export final results
df_final.to_csv(OUTPUT_FILENAME_FINAL_MAX_DRIVE, index=False, encoding='utf-8')

print("\n--- Processing Complete ---")
print(f"Final data (with 3 Similarity Scores) saved to: {OUTPUT_FILENAME_FINAL_MAX_DRIVE}")

Note: At this point, the analysis of the QA model and similarity model has been completed. To facilitate subsequent data analysis, the dataset will be simplified, retaining only the necessary and useful columns and discarding unnecessary data to reduce the dataset file size.

In [ ]:
import pandas as pd
import os # Used for file operations

# --- Assume Drive_PATH and PROJECT_FOLDER are defined in previous code ---
DRIVE_PATH = '/content/drive/MyDrive/'
PROJECT_FOLDER = 'Colab Notebooks/'
# ---------------------------------------------------------------

# 1. Define input and output file paths
INPUT_FILENAME_DRIVE = DRIVE_PATH + PROJECT_FOLDER + 'final_analysis_with_similarity_wide.csv'
OUTPUT_FILENAME_SIMPLIFIED_DRIVE = DRIVE_PATH + PROJECT_FOLDER + 'Most_Simplified_analysis_results.csv'

print(f"Input file path set to: {INPUT_FILENAME_DRIVE}")
print(f"Simplified output file path set to: {OUTPUT_FILENAME_SIMPLIFIED_DRIVE}")

# 2. Define target columns
FINAL_COLUMNS = [
    'text', 'name', 'city',

    # Taste Aspect
    'Taste_Aspect_Answer',
    'Taste_Aspect_Similarity_Score',

    # Service Aspect
    'Service_Aspect_Answer',
    'Service_Aspect_Similarity_Score',

    # Ambiance Aspect
    'Ambiance_Aspect_Answer',
    'Ambiance_Aspect_Similarity_Score'
]

# 3. Read input file
try:
    # Only read the columns we need to save memory
    df_results = pd.read_csv(INPUT_FILENAME_DRIVE, encoding='utf-8')

except FileNotFoundError:
    print(f"\nError: File not found at {INPUT_FILENAME_DRIVE}. Please check the path.")
    # Check if Drive is mounted or the path is correct
    if not os.path.exists(DRIVE_PATH):
        print("Hint: Google Drive does not seem to be mounted successfully. Please run drive.mount('/content/drive').")
    exit()


# 4. Filter DataFrame size, only keeping required columns
# Check which target columns actually exist in the loaded DataFrame
columns_to_keep = [col for col in FINAL_COLUMNS if col in df_results.columns]

if len(columns_to_keep) < 9:
    print(f"\nWarning: Could not find all 9 target columns. Only {len(columns_to_keep)} columns were found.")

# Filter and order by the sequence defined in FINAL_COLUMNS
df_simplified = df_results[columns_to_keep]

# 5. Rename columns to reflect the 'Food' aspect meaning
df_simplified = df_simplified.rename(columns={
    'Taste_Aspect_Answer': 'Food_Aspect_Answer',
    'Taste_Aspect_Similarity_Score': 'Food_Aspect_Similarity_Score'
})

# 6. Export simplified results
df_simplified.to_csv(OUTPUT_FILENAME_SIMPLIFIED_DRIVE, index=False, encoding='utf-8')

print("\n--- Processing Complete ---")
print(f"File read and filtered successfully.")
print(f"Simplified results (containing only {len(columns_to_keep)} columns) saved to: {OUTPUT_FILENAME_SIMPLIFIED_DRIVE}")

Note: At this point, we have a relatively concise instruction set. However, the original dataset is still enormous. To further refine the data and make subsequent analysis more effective, we decided to include only the top 10% of answers with the highest question-answer similarity scores in the analysis.

Before this, since our analysis focuses on English text, we removed non-English text and stipulated that valid text must contain at least 3 English words.

In [ ]:
# install langdetect library
%pip install langdetect
from langdetect import detect, DetectorFactory

tqdm.pandas(desc="Language Detection Progress")

INPUT_FILENAME_DRIVE = DRIVE_PATH + PROJECT_FOLDER + 'Most_Simplified_analysis_results.csv'
OUTPUT_FILENAME_DRIVE = DRIVE_PATH + PROJECT_FOLDER + 'Filtered_English_Texts.csv'

# -------------------------------------------------------------------
# Step 1: Load dataset
# -------------------------------------------------------------------
try:
    # Name the loaded data df_to_analyze to match your filtering code logic
    df_to_analyze = pd.read_csv(INPUT_FILENAME_DRIVE)
    print(f"\nData loaded successfully! Total rows: {len(df_to_analyze)}")
except FileNotFoundError:
    print(f"\nError: File not found. Please check the path: {INPUT_FILENAME_DRIVE}")
    exit()
except Exception as e:
    print(f"\nError occurred while loading file: {e}")
    exit()


# -------------------------------------------------------------------
# Step 2: Define language detection function
# -------------------------------------------------------------------

def safe_detect_language(text):
    """Attempt to detect text language, return 'unknown' on failure"""
    # Ensure text is a non-empty string
    if pd.isna(text) or not isinstance(text, str) or not text.strip():
         return 'na'
    try:
        return detect(text)
    except:
        # Catch text whose language cannot be detected (usually too short or contains non-Latin characters)
        return 'unknown'

print("\n--- Starting data cleaning and filtering ---")
print(f"Data volume before cleaning: {len(df_to_analyze)}")


# -------------------------------------------------------------------
# Step 3: Language filtering (keep English 'en')
# -------------------------------------------------------------------

# 1. Create new column and detect language
# Use .progress_apply to enable progress bar
df_to_analyze['language'] = df_to_analyze['text'].progress_apply(safe_detect_language)

# 2. Filter to keep only English text
df_to_analyze_english = df_to_analyze[df_to_analyze['language'] == 'en'].copy()

print(f"Data volume after filtering non-English: {len(df_to_analyze_english)}")

# Check if any data remains
if len(df_to_analyze_english) == 0:
    print("Warning: No English text remained after filtering, script stopped.")
    exit()


# -------------------------------------------------------------------
# Step 4: Length filtering (word count > 3)
# -------------------------------------------------------------------

# Filtering logic: Use spaces to split text, calculate word count, and require word count > 3
# This step does not require a progress bar as it is a simple string operation
df_filtered = df_to_analyze_english[
    df_to_analyze_english['text'].apply(lambda x: len(str(x).split()) > 3)
].copy()

print(f"Final data volume after filtering length ≤ 3 words: {len(df_filtered)}")


# -------------------------------------------------------------------
# Step 5: Save final results
# -------------------------------------------------------------------

print(f"\n--- Step 5: Save final filtered results to Drive ---")

# Save the final filtered dataset
df_filtered.to_csv(OUTPUT_FILENAME_DRIVE, index=False, encoding='utf-8')

print(f"Filtered data successfully saved to Google Drive:")
print(f"Output file path is: {OUTPUT_FILENAME_DRIVE}")

*** The following will extract the first 10% of the data. ***

In [ ]:
INPUT_FILENAME_DRIVE = DRIVE_PATH + PROJECT_FOLDER + 'Filtered_English_Texts.csv'

# -------------------------------------------------------------------
# Step 2: Load dataset
# -------------------------------------------------------------------
# Assuming the dataset uses comma delimiters
try:
    df = pd.read_csv(INPUT_FILENAME_DRIVE)
    print(f"\nData loaded successfully! Total rows: {len(df)}")
except FileNotFoundError:
    print(f"\nError: File not found. Please check the path: {INPUT_FILENAME_DRIVE}")
    # Stop if file is not found
    exit()
except Exception as e:
    print(f"\nError occurred while loading file: {e}")
    exit()

# -------------------------------------------------------------------
# Step 3: Extract the top 10% of data for the three aspects
# -------------------------------------------------------------------
# Define column names for the three aspect similarity scores
score_columns = [
    'Food_Aspect_Similarity_Score',
    'Service_Aspect_Similarity_Score',
    'Ambiance_Aspect_Similarity_Score'
]

# Dictionary to store the three resulting DataFrames
top_10_data = {}

print("\n--- Starting calculation and filtering for Top 10% data ---")

for col in score_columns:
    # 1. Calculate the Top 10% threshold (90th percentile)
    if col not in df.columns:
        print(f"Error: Column name '{col}' does not exist in the dataset, skipping this aspect.")
        continue

    threshold = df[col].quantile(0.9)
    print(f"Aspect '{col}' Top 10% threshold is: {threshold:.4f}")

    # 2. Filter: Select all rows where the score is greater than or equal to the threshold
    filtered_df = df[df[col] >= threshold].copy()

    # 3. Store the result, and rename columns for distinction when saving
    # Note: The .rename() here is commented out and will keep the original column names
    result_df = filtered_df#.rename(columns={col: 'Final_Similarity_Score'})

    # 4. Add a column to identify the aspect
    aspect_name = col.split('_')[0] # Extract 'Food', 'Service', or 'Ambiance'
    result_df['Aspect'] = aspect_name

    top_10_data[aspect_name] = result_df

    print(f"Aspect '{aspect_name}' extracted {len(filtered_df)} rows of data.")

# -------------------------------------------------------------------
# Step 4: Save results separately to the specified output path (Modified)
# -------------------------------------------------------------------

print("\n--- Step 4: Save the Top 10% results for each aspect separately ---")

saved_files = []
for aspect, df_aspect in top_10_data.items():
    # Construct a unique output filename: e.g., Top_10%_Food_results.csv
    output_filename = f'English_Top_10%_{aspect}_results.csv'
    output_filepath = DRIVE_PATH + PROJECT_FOLDER + output_filename

    # Save DataFrame
    df_aspect.to_csv(output_filepath, index=False, encoding='utf-8')
    saved_files.append(output_filepath)
    print(f"Aspect '{aspect}' with {len(df_aspect)} rows of data saved to: {output_filepath}")

print(f"\nAll Top 10% data has been saved separately into {len(saved_files)} files.")

Note: At this point, all the effective data has been extracted. The final step is sentiment analysis.

In [ ]:
from tqdm.auto import tqdm
import time

# Define the three aspects to be processed
ASPECTS = ['Food', 'Service', 'Ambiance']
BASE_FILENAME = 'English_Top_10%_{}_results.csv'

# --- Model Initialization ---
MODEL_NAME = 'nlptown/bert-base-multilingual-uncased-sentiment'

print("\n--- Sentiment Analysis Model Initialization ---")

# Try to use GPU (device ID 0), otherwise use CPU (-1)
device = 0 if torch.cuda.is_available() and torch.cuda.device_count() > 0 else -1
print(f"Device in use: {'GPU' if device == 0 else 'CPU'}")

try:
    classifier = pipeline(
        'sentiment-analysis',
        model=MODEL_NAME,
        device=device
    )
    print("Model loaded successfully.")
except Exception as e:
    print(f"Model loading failed. Please check if transformers and torch libraries are installed: {e}")
    exit()

# -------------------------------------------------------------------
# Step 2: Loop through and analyze each aspect file
# -------------------------------------------------------------------

for aspect in ASPECTS:
    print(f"\n==================== Starting processing aspect: {aspect} ====================")

    # Construct input and output file paths
    input_file = DRIVE_PATH + PROJECT_FOLDER + BASE_FILENAME.format(aspect)
    # New output file to distinguish it from the original Top 10% file
    output_file = DRIVE_PATH + PROJECT_FOLDER + f'English_Top_10%_{aspect}_sentiment.csv'

    # A. Load Data
    try:
        df_aspect = pd.read_csv(input_file)
        if 'text' not in df_aspect.columns:
             print(f"FATAL ERROR: Missing 'text' column in file '{input_file}'. Skipping.")
             continue
        print(f"File loaded successfully. Data size: {len(df_aspect)} records.")
    except FileNotFoundError:
        print(f"Error: Input file not found: {input_file}. Skipping this aspect.")
        continue
    except Exception as e:
        print(f"Error occurred while loading file: {e}. Skipping this aspect.")
        continue

    # B. Perform Batch Sentiment Analysis (with progress bar)
    texts_to_analyze = df_aspect['text'].tolist()
    sentiment_results = []
    total_texts = len(texts_to_analyze)
    BATCH_SIZE = 32

    print(f"Performing batch sentiment analysis on {total_texts} texts...")

    # Use tqdm loop for batch inference
    for i in tqdm(range(0, total_texts, BATCH_SIZE), desc=f"{aspect} Sentiment Analysis Progress"):
        batch = texts_to_analyze[i:i + BATCH_SIZE]
        results_batch = classifier(batch)
        sentiment_results.extend(results_batch)

    print("Sentiment analysis complete. Integrating results...")

    # C. Integrate Results
    df_aspect['Sentiment_Label'] = [res['label'] for res in sentiment_results]
    df_aspect['Sentiment_Confidence'] = [res['score'] for res in sentiment_results]

    # Print result example
    print(df_aspect[['text', 'Sentiment_Label', 'Sentiment_Confidence']].head())

    # D. Save final results to Drive
    df_aspect.to_csv(output_file, index=False, encoding='utf-8')

    print(f"\nResults successfully saved to new file: {output_file}")

print("\n\n==================== Done. ====================")